In the data analysis phase we identify the query and MessageAttributes columns as candidates for generating additional features.
For each row containing a query, we will break the query into key value pairs and add the keys as columns and the values as cells.
For the MessageAttributes we will extract request and response payload values and add them as new or existing columns (depending on the values).

First we break the SQL query and create more features

In [2]:
import os
import case_id_assignment.utilities as util
import case_id_assignment.feature_engineering as fe
data_folder = '../data'
processed_data_folder = '../processed_data'

In [3]:
isolated_data_path = os.path.join(data_folder, 'ptp_isolated_data.csv')
df = util.load_data_set(isolated_data_path)
df = fe.generate_features_from_sql(df)

Convert SQL Queries into additional columns:   0%|          | 0/26006 [00:00<?, ?it/s]

finished processing the data, create data frames
merge data frames to one global data frame


 Then we create more features from the MessageAttributes

In [ ]:
df_processed = fe.generate_features_from_http(data_set=df)
df_processed.info()

Next part is imputation of the data - we will use hot deck imputation

In [4]:
processed_data_path = os.path.join(processed_data_folder, 'isolated_df_processed.csv')
p_df = util.load_data_set(processed_data_path)

C:\Research\cade-id-assignment\src\case_id_assignment\utilities.py:17: DtypeWarning: Columns (27,33,35,36,61,63,69,77,78,91,108,109,110,111,112,113,116,119,122,123,126,127,128,129,130,132,133,134,136,137,140,141,142,144,145,148,149,152,154,155,157,158,160,161,166,174,179,186,189,190,191,192,199,202,205,206,208,211,213,215,230,231,232,234,235) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [5]:
p_df.head()

,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,...,multi,payment_date,payment_difference_handling,payment_method_id,payment_type,writeoff_label,payment_term_id,request_method_call,starting_frame_number,file_data
0,0,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.637118,119,2020-12-22 11:39:44.237118,2020-12-22 11:39:44.237118,192.168.11.2-192.168.11.9 (1),http,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,version,NaN,['version']
1,1,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.688344,153,2020-12-22 11:48:16.548344,2020-12-22 11:48:16.548344,192.168.11.2-192.168.11.9 (1),http,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,server_version,119.0,"['server_version', '12.0-20190820', 'server_ve..."
2,2,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.690577,161,2020-12-22 11:48:38.880577,2020-12-22 11:48:38.880577,192.168.11.2-192.168.11.9 (2),http,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,authenticate,NaN,"['authenticate', 'odoo01', 'user.suername@comp..."
3,3,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.985813,224,2020-12-22 12:37:51.535813,2020-12-22 12:37:51.535813,192.168.11.1-192.168.11.2 (7),pgsql,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,4,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.994598,245,2020-12-22 12:39:19.394598,2020-12-22 12:39:19.394598,192.168.11.2-192.168.11.9 (2),http,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,161.0,['2']


In [7]:
column_for_search = p_df.columns[25]
column_for_search

'write_date'

In [18]:
p_df[['frame.number','HighestLayerProtocol','MessageAttributes']][:6]

,frame.number,HighestLayerProtocol,MessageAttributes
0,119,http,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '..."
1,153,http,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E..."
2,161,http,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '..."
3,224,pgsql,"{'pgsql.type': 'Simple query', 'pgsql.length':..."
4,245,http,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E..."
5,253,http,"{'': 'POST /xmlrpc/2/object HTTP/1.1\\r\\n', '..."


In [12]:
p_df["Indexes"]= p_df["MessageAttributes"].str.find(column_for_search)

In [13]:
p_df["Indexes"]

0         -1
1         -1
2         -1
3        171
4         -1
        ... 
26001     -1
26002     -1
26003    170
26004     -1
26005    169
Name: Indexes, Length: 26006, dtype: int64